# Este script baja imagen segun un aoi dibujado interactivamente, y luego, clipea con el aoi (sobre la imagen ya bajada)
* busca imagenes por filtros
* genera un calculo de overlapping con el aoi determinado
* filtra de la base total aquellas imagenes que cumplan con un criterio (por ej, :x% overlaping)
* activa el asset que se quiera, y baja las imagenes zipeadas en un directorio a definir


In [ ]:
import sys
import os
import json
import scipy
import urllib
import datetime 
import urllib3
import rasterio
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
from osgeo import gdal
from planet import api
from planet.api import filters
from traitlets import link
import rasterio.mask as rio_mask
from shapely.geometry import mapping, shape
from IPython.display import display, Image, HTML
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
urllib3.disable_warnings()
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

api_keys = json.load(open("apikeys.json",'r'))
client = api.ClientV1(api_key=api_keys["PLANET_API_KEY"])


# Make a slippy map to get GeoJSON
The planet API allows you to query using a geojson which is a special flavor of json.
We are going to create a slippy map using leaflet and apply the Planet 2017 Q1 mosaic as the basemap. This requires our api key.
We are going to add a special draw handler that shoves a draw region into a object so we get the geojson.
If you don't want to do this, or need a fixed query try geojson.io
To install and run:
$ pip install ipyleaflet
$ jupyter nbextension enable --py --sys-prefix ipyleaflet
$ jupyter nbextension enable --py --sys-prefix widgetsnbextension
More information

## filtros y definir aoi

In [ ]:
myAOI = json.load(open("estanzuela.json",'r'))
#print(myAOI)
# build a query using the AOI and
# a cloud_cover filter that excludes 'cloud free' scenes

old = datetime.datetime(year=2017,month=10,day=1)

query = filters.and_filter(
    filters.geom_filter(myAOI),
    filters.range_filter('cloud_cover', lt=10),
    filters.date_range('acquired', gt=old)
)

# build a request for only PlanetScope imagery
request = filters.build_search_request(
    query, item_types=['PSScene4Band']
)

# if you don't have an API key configured, this will raise an exception
result = client.quick_search(request)
#print(result.items_iter)
scenes = []
planet_map = {}
for item in result.items_iter(limit=500):
    planet_map[item['id']]=item
    props = item['properties']
    props["id"] = item["id"]
    props["geometry"] = item["geometry"]
    props["thumbnail"] = item["_links"]["thumbnail"]
    #print(props)
    scenes.append(props)
    #print(scenes)
scenes = pd.DataFrame(data=scenes)
display(scenes)
print(len(scenes))

# Cleanup
The data we got back is good, but we need some more information
We got back big scenes, but we only care about our area of interest. The scene may not cover the whole area of interest.
We can use the Shapely library to quickly figure out how much each scene overlaps our AOI
We will convert our AOI and the geometry of each scene to calculate overlap using a shapely call.
The returned acquisition, publish, and update times are strings, we'll convert them to datatime objects so we wan search.

In [ ]:
# now let's clean up the datetime stuff
# make a shapely shape from our aoi
AOI = shape(myAOI)
footprints = []
overlaps = []
# go through the geometry from our api call, convert to a shape and calculate overlap area.
# also save the shape for safe keeping
for footprint in scenes["geometry"].tolist():
    s = shape(footprint)
    footprints.append(s)
    overlap = 100.0*(AOI.intersection(s).area / AOI.area)
    overlaps.append(overlap)
# take our lists and add them back to our dataframe
scenes['overlap'] = pd.Series(overlaps, index=scenes.index)
scenes['footprint'] = pd.Series(footprints, index=scenes.index)
# now make sure pandas knows about our date/time columns.
scenes["acquired"] = pd.to_datetime(scenes["acquired"])
scenes["published"] = pd.to_datetime(scenes["published"])
scenes["updated"] = pd.to_datetime(scenes["updated"])
scenes.head()

# Filtering our search using pandas.
* Using our dataframe we will filter the scenes to just what we want.
* First we want scenes with less than 10% clouds.
* Second we want standard quality images. Test images may not be high quality.
* Third well only look for scenes since January.
* Finally we will create a new data frame with our queries and print the results.

In [ ]:
# Now let's get it down to just good, recent, clear scenes
clear = scenes['cloud_cover']<0.1
good = scenes['quality_category']=="standard"
recent = scenes["acquired"] > datetime.date(year=2018,month=1,day=1)
partial_coverage = scenes["overlap"] >90
good_scenes = scenes[(good&clear&recent&partial_coverage)]
display(good_scenes)
print (len(good_scenes))

# Now let's get it down to just good, recent, clear scenes
clear = scenes['cloud_cover']<0.5
good = scenes['quality_category']=="standard"
all_time = scenes["acquired"] > datetime.date(year=2017,month=11,day=1)
full_coverage = scenes["overlap"] >= 60
all_scenes = scenes[(good&clear&all_time&full_coverage)]
#display(all_scenes)
print (len(all_scenes))

# Product Activation and Downloading
There are two things we need to know, the satellite type (asset) and image type (product).
Full resolution uncompressed satellite images are big and there are lots of ways to view them.
For this reason Planet generally keeps images in their native format and only processes them on customer requests. There is some caching of processed scenes, but this is the exception not the rule.
All images must be activated prior to downloading and this can take some time based on demand.
Additionally we need to determine what sort of product we want to download. Generally speaking there are three kinds of scenes:
Analytic - multi-band full resolution images that have not been processed. These are like raw files for DSLR camers.
Visual - these are color corrected rectified tifs. If you are just starting out this is your best call.
UDM - Usable data mask. This mask can be used to find bad pixels and columns and to mask out areas with clouds.

In [ ]:
#good_scenes=pd.Series.tolist(good_scenes)
print(good_scenes)

In [ ]:
import requests
import time

# Set Item Type
item_type = 'PSScene4Band'

# Set Asset Type
asset_type = 'analytic_sr'

#lista de los nombres de imagenes a bajar (era scene_id)
to_get=all_scenes['id'].tolist()
print(to_get)
#calcular el area de la sumatoria de las imagenes#
#
api_key=api_keys["PLANET_API_KEY"]
url_base=[]
i=0
for scenes in to_get[0:3]:
# Request clip of scene (This will take some time to complete)
    #calcular el area de la seleccion de imagenes hecha e imprimir#
    
    clip_payload = {'aoi': myAOI,'targets': [{'item_id': to_get[i],'item_type': item_type,'asset_type': asset_type}]}
    #print(clip_payload)
    i=i+1
    request = requests.post('https://api.planet.com/compute/ops/clips/v1', auth=(api_key, ''), json=clip_payload)
    print(request)
    clip_url= request.json()['_links']['_self']
    print(clip_url)
    # Poll API to monitor clip status. Once finished, download and upzip the scene
    clip_succeeded = False
    while not clip_succeeded:

    # Poll API
        check_state_request = requests.get(clip_url, auth=(api_key, ''))
        #print(check_state_request)
    # If clipping process succeeded , we are done
        if check_state_request.json()['state'] == 'succeeded':
            clip_download_url = check_state_request.json()['_links']['results'][0]
            clip_succeeded = True
            #print(clip_download_url)
            if clip_download_url in url_base: continue
            url_base.append(clip_download_url)
            print("Clip of scene succeeded and is ready to download") 
    
    # Still activating. Wait 1 second and check again.
        else:
            print("...Still waiting for clipping to complete...")
            time.sleep(2)

print(len(url_base))

Download and upzip the clip
Once complete, look in the output directory to see your clipped tif file.

NOTE: Clipped scene will only be available for 5 minutes.

In [ ]:
import os
from tqdm import tqdm
import zipfile
from datetime import datetime

# ubicar los archivos donde se desee
path_or='D:/javie/Descargas/'

print(to_get[0:3])
#print(url_base)
i=0    
downloaded=[]
for url in url_base[0:2]: 
    response = requests.get(url, stream=True)
    #print(response)
    scene_id=to_get[i]
    print(scene_id)
    path =os.path.join(path_or+ scene_id +'_'+ asset_type +'_'+ myAOI['name'])
    #print(path)
    i= i+1
    if os.path.isfile(path + '.zip' ):
        print ("skipping, we allready have scene: ", scene_id)
        continue 
    else:
        with open(path +'.zip', "wb") as handle:
            for data in tqdm(response.iter_content()):
                #print(data)
                handle.write(data)
        downloaded.append(path)
        print(downloaded)

        date=datetime.now()
        date=date.strftime('%Y_%m_%d_%H%M')

        newpath=os.path.join(path_or, date + '_downloaded.txt')
        #print(newpath)
        with open (newpath , "w") as pronto:
            for item in downloaded:
                pronto.write("%s\n" % item)       
    #Unzip file
        ziped_item = zipfile.ZipFile(path + ".zip")
        ziped_item.extractall(path)     
  
    # Delete zip file
        #time.sleep(5)
        #os.remove(path + '.zip')
        print('Downloaded clips located in:', path_or)    
